In [22]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import time
import os

- tensorflow/keras: Para construir y entrenar redes neuronales
- numpy: Para operaciones matemáticas con arrays
- time: Para medir duración del entrenamiento
- os: Para operaciones del sistema de archivos


In [23]:
configuracion_inicial = {
    'batch_size': 32,
    'epocas': 10,
    'split_validacion': 0.2,
    'r_seed': 42
}


**configuracion_inicial**: Diccionario con parámetros fijos para todos los experimentos

- **batch_size**: Cada 32 imagenes procesadas, se calcula el error promedio y se ajustan los pesos. Los pesos son parametros internos de la red neuronal que determinan su importancia

- **epocas**: Número de veces que el algoritmo pasa por el dataset completo

- **split_validacion**: Fracción del dataset que se separa para validación. La validación sirve para evaluar el modelo en datos no vistos durante el entrenamiento y evitar que memorize los datos del entrenamiento.

- **r_seed**: Semilla para generar numeros aleatorios. Iniciliza los pesos, podria ser cualquier número.


---


Imágenes de entrenamiento: 48,000 (después de split_validacion: 20%)

batch_size: 32

Batches por epoca: 48,000 ÷ 32 = 1,500 batches

Actualizaciones por epoca: 1,500 actualizaciones de pesos

epocas: 10

Actualizaciones totales: 1,500 × 10 = 15,000 actualizaciones



In [24]:
configuracion_de_optimizadores = {
    'SGD': {
        'class': keras.optimizers.SGD,
        'params': {'learning_rate': 0.01}  # Tamaño del paso para actualizar pesos
    },
    'Adam': {
        'class': keras.optimizers.Adam,
        'params': {'learning_rate': 0.001}  # Tasa de aprendizaje más pequeña
    }
}


SGD: Primer optimizador: Stochastic Gradient Descent

Adam: Segundo optimizador: Adaptive Moment Estimation




---




DIFERENCIA ENTRE SGD Y ADAM:

- SGD con learning_rate=0.01:

- Pasos más grandes: 0.01 es relativamente grande

- Actualizaciones más agresivas: Los pesos cambian más en cada paso

- Razón: SGD es un algoritmo simple que necesita pasos más grandes para converger

- Adam con learning_rate=0.001:

- Pasos más pequeños: 0.001 es 10 veces más pequeño que SGD

- Actualizaciones más conservadoras: Los pesos cambian menos en cada paso

- Razón: Adam es más inteligente y se adapta automáticamente, por eso funciona mejor con pasos pequeños





In [25]:
def cargar_datos():

    # Descarga
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    #Normalización
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0

    # Dimensionar
    x_train = x_train.reshape(-1, 28, 28, 1)
    x_test = x_test.reshape(-1, 28, 28, 1)

    return (x_train, y_train), (x_test, y_test)

# Guardar datos
(x_train, y_train), (x_test, y_test) = cargar_datos()

**¿Como funciona la "Descarga"?**

x_train: 60,000 imágenes de entrenamiento (28x28 píxeles)

y_train: 60,000 etiquetas (números del 0-9) que indican qué dígito es cada imagen

x_test: 10,000 imágenes para testing

y_test: 10,000 etiquetas para testing



---

**¿Como funciona la "Normalizacion"?**

Se convierten los valores enteros (0-255) a decimales y luego se dividen en 255 para que se encuentren en el rango de 0-1.
Esto se hace porque las redes neuronales aprenden mejor con valores pequeños.



---

**¿Como funciona el apartado de "Dimensionar"?**
El codigo cambia la forma de los arrays para que sean compatibles con capas convolucionales

Forma original: (60000, 28, 28) - 60,000 imágenes de 28×28 píxeles

Forma nueva: (60000, 28, 28, 1) - 60,000 imágenes de 28×28×1 píxeles

**El 1 final** le dice a la red neuronal que las imagenes son en blanco y negro, no en escala RGB, cada pixel tiene una escala de brillo en la escala de grises. Keras requiere esto.



---

**Guardado de datos**
Se guardan los datos en variables

x_train: Las imágenes para entrenar (60,000)

y_train: Las respuestas correctas de las imágenes de entrenamiento

x_test: Las imágenes para probar (10,000)

y_test: Las respuestas correctas de las imágenes de prueba

In [26]:
def crear_modelo():

    model = keras.Sequential([
        # CAPA 1: Convolucional (Conv2D)
        keras.layers.Conv2D(
            filters=32,
            kernel_size=(3, 3),
            activation='relu',
            input_shape=(28, 28, 1)
        ),

        # CAPA 2: MAX Pooling
        keras.layers.MaxPooling2D(pool_size=(2, 2)),

        # CAPA 3: Convolucional 2da Capa
        keras.layers.Conv2D(
            filters=64,
            kernel_size=(3, 3),
            activation='relu'
        ),

        # CAPA 4: MAX Pooling 2da Capa
        keras.layers.MaxPooling2D(pool_size=(2, 2)),

        # CAPA 5: Aplanar
        keras.layers.Flatten(),

        # CAPA 6: Densa
        keras.layers.Dense(128, activation='relu'),

        # CAPA 7: Salida
        keras.layers.Dense(10, activation='softmax')
    ])

    return model

EjemploModelo = crear_modelo()
EjemploModelo.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225,034 (879.04 KB)

 Trainable params: 225,034 (879.04 KB)

 Non-trainable params: 0 (0.00 B)

Esta función construye una red neuronal convolucional (CNN) específicamente diseñada para clasificar imágenes de dígitos escritos a mano del dataset MNIST




# **CAPA 1: Convolucional (Conv2D)**


- Aplica 32 filtros diferentes sobre la imagen

- Cada filtro es una matriz de 3×3 píxeles que se desliza por toda la imagen

- Detecta características locales como bordes, esquinas, curvas

**Parámetros específicos:**

filters=32: Crea 32 mapas de características diferentes

kernel_size=(3, 3): Cada filtro examina áreas de 3×3 píxeles

activation='relu': Función que devuelve el valor si es positivo, 0 si es negativo

input_shape=(28, 28, 1): Especifica que entran imágenes 28×28 en escala de grises


#**CAPA 2: Max Pooling**


- Reduce la dimensionalidad tomando el valor máximo en ventanas de 2×2

- Transforma imágenes de 28×28 a 14×14

- Mantiene las características más importantes mientras reduce el tamaño



#**CAPA 3: Convolucional (Segunda capa)**


- Aplica 64 filtros sobre los mapas de características de la capa anterior

- Detecta patrones más complejos combinando características simples




#**CAPA 4: Max Pooling (Segunda capa)**


- Reduce de 14×14 a 7×7


#**CAPA 5: Aplanar**

- Convierte la salida 3D (7, 7, 64) en un vector 1D de 3,136 elementos

- Prepara los datos para las capas densas (completamente conectadas)

- Sin esta capa, las capas densas no podrían procesar los datos


#**CAPA 6: Densa**

- 128 neuronas donde cada una se conecta a todas las 3,136 entradas

- Combina todas las características extraídas por las capas convolucionales

- Aprende relaciones complejas entre las diferentes características

#**CAPA 7: Salida (Output)**

- 10 neuronas de salida - una para cada dígito posible (0-9)

- activation='softmax': Convierte las salidas en probabilidades que suman 1.0

Ejemplo: [0.02, 0.85, 0.01, 0.03, 0.02, 0.01, 0.02, 0.01, 0.02, 0.01] = 85% de probabilidad de que sea el dígito "1"


In [27]:
def entrenar_optimizador(nombre_optimizador, optimizador):

    #Creacion
    model = crear_modelo()

    #Configuracion
    model.compile(
        optimizer=optimizador,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # Tiempos
    TiempoInicio = time.time()

    #Entrenamiento
    historial = model.fit(
        x_train,
        y_train,
        epochs=configuracion_inicial['epocas'],
        batch_size=configuracion_inicial['batch_size'],
        validation_split=configuracion_inicial['split_validacion'],
        verbose=0,
        shuffle=True
    )

    #Calculo del tiempo
    TiempoTotal = time.time() - TiempoInicio

    # Resultados
    resultados = {
        'nombre': nombre_optimizador,
        'historial': historial.history,
        'tiempo_segundos': TiempoTotal,
        'modelo': model
    }

    #Informacion
    print(f" {nombre_optimizador} completado")
    print(f"   Tiempo: {TiempoTotal:.2f} segundos")
    print(f"   Precisión final: {historial.history['val_accuracy'][-1]:.4f}")

    return resultados


Esta función entrena una red neuronal completa usando un optimizador específico (SGD o Adam) y devuelve todos los resultados del entrenamiento para su análisis



**Parametros de entrada**

- **nombre_optimizador**: String con el nombre ("SGD" o "Adam") para identificar el experimento

- **optimizador**: Objeto de Keras que contiene el algoritmo de optimización configurado



#**Configuracion**

**optimizer=optimizador:**

- Configura el algoritmo de optimización que ajustará los pesos

- Puede ser SGD, Adam, u otro definido en ""configuracion_de_optimizadores""


---


**loss='sparse_categorical_crossentropy':**

- Función de perdida que mide que tan incorrectas son las predicciones

- Mientras mas baja sea la perdida, mejor sera el modelo



---


**metrics=['accuracy']:**

- Métrica de evaluación: % de predicciones correctas

- Se calcula como: (Numero de aciertos / todas las muestras) × 100

#**Tiempos**

Toma una marca de tiempo antes de comenzar el entrenamiento y permite calcular cuanto ha pasado desde que se inicio el proceso.

#**Entrenamiento**

**x_train, y_train:

Datos de entrenamiento: 60,000 imágenes y sus etiquetas correctas


---


**epocas=10:**

- Cantidad de vueltas completas por todo el dataset de entrenamiento

- En cada época, el modelo "revisa" todas las imágenes una vez


---


**batch_size=32:**

- Cantidad de imágenes procesadas antes de cada actualización de los pesos

- 48,000 imágenes ÷ 32 = 1,500 actualizaciones por epoca



---


**split_validacion=0.2:**

- Separa automáticamente 20% de los datos para validación

- 48,000 imágenes para entrenamiento, 12,000 para validación


---


**verbose=0:**

- Funcion para desactivar cualquier output DURANTE el entrenamiento


---

**shuffle=True:**

- Mezcla los datos antes de cada epoca para evitar que el modelo aprenda patrones en el orden de los datos

#**Calculo del tiempo**
- Toma la marca de tiempo actual y resta la del tiempo inicial lo cual nos resulta en cuanto tiempo demoro el entrenamiento en segundos

#**Resultados**
**historial.history**

- **accuracy**: Precisión en entrenamiento por cada epoca

- **val_accuracy**: Precisión en la etapa de validacion por cada epoca

- loss: Perdida en entrenamiento por cada epoca

- val_loss: Perdida en validación por cada época

**modelo**:

- Se guarda el modelo completo

#**Informacion**

- Nombre del optimizador

- Tiempo total en segundos con 2 decimales

- Precisión final en validación de la ultima epoca con 4 decimales



In [28]:
def comparar_optimizadores():

    #Inicilizacion
    resultados = {}

    #Iteraciones sobre optimizadores
    for nombre, config in configuracion_de_optimizadores.items():

        #Creacion de optimizadores
        optimizador = config['class'](**config['params'])

        #Entrenamiento
        resultado = entrenar_optimizador(nombre, optimizador)

        #Guardado
        resultados[nombre] = resultado

    return resultados

#Resultados
resultados_comparacion = comparar_optimizadores()

 SGD completado
   Tiempo: 666.43 segundos
   Precisión final: 0.9835
 Adam completado
   Tiempo: 538.03 segundos
   Precisión final: 0.9908


Ejecuta la comparación completa entre SGD y Adam entrenando un modelo con cada optimizador bajo las mismas condiciones.

#**Inicilizacion**
- Crea un diccionario vacío para almacenar todos los resultados

#**Iteraciones sobre optimizadores**
- Recorre cada optimizador definido en la configuración (SGD y Adam)

#**Creacion de optimizadores**

- SGD: keras.optimizers.SGD(learning_rate=0.01)

- Adam: keras.optimizers.Adam(learning_rate=0.001)

Crea la instancia específica con sus parámetros

#**Entrenamiento**
- Llama a la función que entrena el modelo completo

- Pasa el nombre ("SGD" o "Adam") y el optimizador configurado

#**Guardado**
- Guarda los resultados de cada optimizador en el diccionario

#**Resultados**
Retorna un diccionario con:

- resultados['SGD']: Todos los datos del entrenamiento con SGD

- resultados['Adam']: Todos los datos del entrenamiento con Adam

In [30]:
def preparar_datos_para_analisis(resultados):

    datos_para_analisis = {}

    for nombre, resultado in resultados.items():

        datos_para_analisis[nombre] = {
            #Estadisticas por epoca
            'precision_entrenamiento': resultado['historial']['accuracy'],
            'precision_validacion': resultado['historial']['val_accuracy'],
            'perdida_entrenamiento': resultado['historial']['loss'],
            'perdida_validacion': resultado['historial']['val_loss'],

            #Estadisticas especificas
            'tiempo_total_segundos': resultado['tiempo_segundos'],
            'total_epocas': len(resultado['historial']['accuracy']),

            #Optimizador
            'nombre_optimizador': nombre,
            'configuracion': resultado['modelo'].optimizer.get_config()
        }

    return datos_para_analisis

Organiza todos los resultados del entrenamiento en un formato estructurado para que se puedan generar gráficos y análisis


#**Estadisticas por epoca**
- precision_entrenamiento: Precisión en datos de entrenamiento

- precision_validacion: Precisión en datos de validación

- perdida_entrenamiento: Error en datos de entrenamiento

- perdida_validacion: Error en datos de validación

#**Estadisticas especificas**
- tiempo_total_segundos: Duración total del entrenamiento

- total_epocas: Número de épocas entrenadas

#**Optimizador**
- nombre_optimizador: "SGD" o "Adam"

- configuracion: Parametros especificos usados